In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import BooleanType
spark = SparkSession.builder.appName("Ngram_buzzwords").master("spark://192.168.1.76:7077").getOrCreate()
sc = spark.sparkContext

In [2]:
df = spark.read.csv('../data/eng-gb-all-only-letters.csv', sep="\t", header=False)

In [3]:
df = df.withColumnRenamed('_c0', 'ngram')
df = df.withColumnRenamed('_c1', 'year')
df = df.withColumnRenamed('_c2', 'match_count')
df = df.withColumnRenamed('_c3', 'page_count')
df = df.withColumnRenamed('_c4', 'volume_count')

In [4]:
#df.count()

In [5]:
rdd = df.rdd.map(list)

In [6]:
def splitAndMakeNgramKey(row):
    elements = row[0].split(",")
    return(elements[0], (elements[1:len(elements)]))

rdd = rdd.map(splitAndMakeNgramKey)

In [7]:
rdd.take(10)

[('A', ['1541', '8', '7', '1']),
 ('A', ['1547', '112', '77', '1']),
 ('A', ['1551', '40', '30', '1']),
 ('A', ['1562', '378', '196', '1']),
 ('A', ['1576', '41', '26', '1']),
 ('A', ['1582', '17', '17', '1']),
 ('A', ['1584', '15', '13', '1']),
 ('A', ['1588', '51', '38', '1']),
 ('A', ['1590', '43', '26', '1']),
 ('A', ['1591', '25', '20', '1'])]

In [8]:
def groupData(a,b):
    try:
        foo = a[0][0]
    except:
        a = [a]
    return a.append(b)

rdd = rdd.reduceByKey(groupData)

In [ ]:
rdd.take(10)

In [ ]:
#df.coalesce(1).write.csv("eng-gb-all-only-letters")